# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 6 2022 1:57PM,238900,10,MET210931,"Methapharm, Inc.",Released
1,Jun 6 2022 1:57PM,238900,10,MET210942,"Methapharm, Inc.",Released
2,Jun 6 2022 1:57PM,238900,10,MET210948,"Methapharm, Inc.",Released
3,Jun 6 2022 1:57PM,238900,10,MET210950,"Methapharm, Inc.",Released
4,Jun 6 2022 1:57PM,238900,10,MET210951,"Methapharm, Inc.",Released
5,Jun 6 2022 1:57PM,238900,10,MET210952,"Methapharm, Inc.",Released
6,Jun 6 2022 1:57PM,238900,10,MET210953,"Methapharm, Inc.",Released
7,Jun 6 2022 1:57PM,238900,10,MET210954,"Methapharm, Inc.",Released
8,Jun 6 2022 1:57PM,238900,10,MET210956,"Methapharm, Inc.",Released
9,Jun 6 2022 1:57PM,238900,10,MET210939,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,238894,Released,1
31,238895,Released,27
32,238896,Released,1
33,238897,Released,1
34,238900,Released,10


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238894,NaN,NaN,1.0
238895,NaN,NaN,27.0
238896,NaN,NaN,1.0
238897,NaN,NaN,1.0
238900,NaN,NaN,10.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238820,5.0,6.0,2.0
238836,0.0,0.0,51.0
238838,0.0,0.0,1.0
238844,0.0,0.0,3.0
238852,0.0,1.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238820,5,6,2
238836,0,0,51
238838,0,0,1
238844,0,0,3
238852,0,1,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238820,5,6,2
1,238836,0,0,51
2,238838,0,0,1
3,238844,0,0,3
4,238852,0,1,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238820,5,6,2
1,238836,,,51
2,238838,,,1
3,238844,,,3
4,238852,,1,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 6 2022 1:57PM,238900,10,"Methapharm, Inc."
10,Jun 6 2022 1:56PM,238897,102,"Senseonics, Incorporated"
11,Jun 6 2022 1:53PM,238895,10,"Methapharm, Inc."
38,Jun 6 2022 1:52PM,238896,10,"Methapharm, Inc."
39,Jun 6 2022 1:46PM,238894,21,"NBTY Global, Inc."
40,Jun 6 2022 1:42PM,238893,10,"Methapharm, Inc."
41,Jun 6 2022 1:37PM,238892,16,Sartorius Stedim North America
42,Jun 6 2022 1:27PM,238891,21,"NBTY Global, Inc."
43,Jun 6 2022 1:24PM,238890,19,"AdvaGen Pharma, Ltd"
48,Jun 6 2022 1:10PM,238889,20,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 6 2022 1:57PM,238900,10,"Methapharm, Inc.",,,10
1,Jun 6 2022 1:56PM,238897,102,"Senseonics, Incorporated",,,1
2,Jun 6 2022 1:53PM,238895,10,"Methapharm, Inc.",,,27
3,Jun 6 2022 1:52PM,238896,10,"Methapharm, Inc.",,,1
4,Jun 6 2022 1:46PM,238894,21,"NBTY Global, Inc.",,,1
5,Jun 6 2022 1:42PM,238893,10,"Methapharm, Inc.",,,1
6,Jun 6 2022 1:37PM,238892,16,Sartorius Stedim North America,,,1
7,Jun 6 2022 1:27PM,238891,21,"NBTY Global, Inc.",,1,
8,Jun 6 2022 1:24PM,238890,19,"AdvaGen Pharma, Ltd",,,5
9,Jun 6 2022 1:10PM,238889,20,"Exact-Rx, Inc.",,,17


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 1:57PM,238900,10,"Methapharm, Inc.",10,,
1,Jun 6 2022 1:56PM,238897,102,"Senseonics, Incorporated",1,,
2,Jun 6 2022 1:53PM,238895,10,"Methapharm, Inc.",27,,
3,Jun 6 2022 1:52PM,238896,10,"Methapharm, Inc.",1,,
4,Jun 6 2022 1:46PM,238894,21,"NBTY Global, Inc.",1,,
5,Jun 6 2022 1:42PM,238893,10,"Methapharm, Inc.",1,,
6,Jun 6 2022 1:37PM,238892,16,Sartorius Stedim North America,1,,
7,Jun 6 2022 1:27PM,238891,21,"NBTY Global, Inc.",,1,
8,Jun 6 2022 1:24PM,238890,19,"AdvaGen Pharma, Ltd",5,,
9,Jun 6 2022 1:10PM,238889,20,"Exact-Rx, Inc.",17,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 1:57PM,238900,10,"Methapharm, Inc.",10,,
1,Jun 6 2022 1:56PM,238897,102,"Senseonics, Incorporated",1,,
2,Jun 6 2022 1:53PM,238895,10,"Methapharm, Inc.",27,,
3,Jun 6 2022 1:52PM,238896,10,"Methapharm, Inc.",1,,
4,Jun 6 2022 1:46PM,238894,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 1:57PM,238900,10,"Methapharm, Inc.",10.0,NaN,NaN
1,Jun 6 2022 1:56PM,238897,102,"Senseonics, Incorporated",1.0,NaN,NaN
2,Jun 6 2022 1:53PM,238895,10,"Methapharm, Inc.",27.0,NaN,NaN
3,Jun 6 2022 1:52PM,238896,10,"Methapharm, Inc.",1.0,NaN,NaN
4,Jun 6 2022 1:46PM,238894,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 1:57PM,238900,10,"Methapharm, Inc.",10.0,0.0,0.0
1,Jun 6 2022 1:56PM,238897,102,"Senseonics, Incorporated",1.0,0.0,0.0
2,Jun 6 2022 1:53PM,238895,10,"Methapharm, Inc.",27.0,0.0,0.0
3,Jun 6 2022 1:52PM,238896,10,"Methapharm, Inc.",1.0,0.0,0.0
4,Jun 6 2022 1:46PM,238894,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2388786,102.0,14.0,0.0
102,238897,1.0,0.0,0.0
12,238886,1.0,0.0,0.0
15,238852,1.0,1.0,0.0
16,238892,1.0,0.0,0.0
17,238873,1.0,0.0,0.0
18,477771,2.0,0.0,0.0
19,955450,14.0,11.0,5.0
20,1194337,127.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2388786,102.0,14.0,0.0
1,102,238897,1.0,0.0,0.0
2,12,238886,1.0,0.0,0.0
3,15,238852,1.0,1.0,0.0
4,16,238892,1.0,0.0,0.0
5,17,238873,1.0,0.0,0.0
6,18,477771,2.0,0.0,0.0
7,19,955450,14.0,11.0,5.0
8,20,1194337,127.0,2.0,0.0
9,21,477785,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,102.0,14.0,0.0
1,102,1.0,0.0,0.0
2,12,1.0,0.0,0.0
3,15,1.0,1.0,0.0
4,16,1.0,0.0,0.0
5,17,1.0,0.0,0.0
6,18,2.0,0.0,0.0
7,19,14.0,11.0,5.0
8,20,127.0,2.0,0.0
9,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,102.0
1,102,Released,1.0
2,12,Released,1.0
3,15,Released,1.0
4,16,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,17,18,19,20,21,22
Status,,,,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
Executing,14.0,0.0,0.0,1.0,0.0,0.0,0.0,11.0,2.0,1.0,0.0
Released,102.0,1.0,1.0,1.0,1.0,1.0,2.0,14.0,127.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
1,Executing,14.0,0.0,0.0,1.0,0.0,0.0,0.0,11.0,2.0,1.0,0.0
2,Released,102.0,1.0,1.0,1.0,1.0,1.0,2.0,14.0,127.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
1,Executing,14.0,0.0,0.0,1.0,0.0,0.0,0.0,11.0,2.0,1.0,0.0
2,Released,102.0,1.0,1.0,1.0,1.0,1.0,2.0,14.0,127.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()